In [1]:
print("My Codespace is working now")

My Codespace is working now


# Calling the SWC API

## Calling the Health Check endpoint with httpx

In [1]:
import httpx

base_url = "https://api.sportsworldcentral.com"
endpoint = "/"

api_response = httpx.get(base_url + endpoint)
print(api_response.status_code)
print(api_response.json())


200
{'message': 'API health check successful'}


## Gathering team rosters

In [2]:
endpoint = "/v0/leagues/"
api_response = httpx.get(base_url + endpoint)
print(api_response.status_code)
print(api_response.json())

200
[{'league_id': 5001, 'league_name': 'Pigskin Prodigal Fantasy League', 'scoring_type': 'PPR', 'last_changed_date': '2024-04-22', 'teams': [{'league_id': 5001, 'team_id': 1007, 'team_name': 'Checkers Not Chess', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1003, 'team_name': 'Dallas Steers', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1008, 'team_name': 'Defending Champs', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1011, 'team_name': 'Double Option', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1005, 'team_name': 'Fighting Catfish', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1010, 'team_name': 'Fumblerooski', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1004, 'team_name': 'Never Gonna Give Up', 'last_changed_date': '2024-04-22'}, {'league_id': 5001, 'team_id': 1002, 'team_name': 'Pigskin Party', 'last_changed_date': '2024-04-22'}, {'league_id': 500

## Formatting Team Rosters


In [13]:
import pandas as pd
data = api_response.json()
df = pd.DataFrame.from_dict(data)


In [15]:
print(df)

   league_id                      league_name scoring_type last_changed_date  \
0       5001  Pigskin Prodigal Fantasy League          PPR        2024-04-22   
1       5002       Recurring Champions League     Half-PPR        2024-04-22   
2       5003                      AHAHFZZFFFL     Standard        2024-04-22   
3       5004    Gridiron Gurus Fantasy League          PPR        2024-04-22   
4       5005                 Best League Ever          PPR        2024-04-22   

                                               teams  
0  [{'league_id': 5001, 'team_id': 1007, 'team_na...  
1  [{'league_id': 5002, 'team_id': 1019, 'team_na...  
2                                                 []  
3                                                 []  
4                                                 []  


In [16]:
normalized_df = pd.json_normalize(df)
print(normalized_df)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


In [17]:
# Extracting league data and team data
leagues = []
teams = []

for league in data:
    league_info = {
        'league_id': league['league_id'],
        'league_name': league['league_name'],
        'scoring_type': league['scoring_type'],
        'last_changed_date': league['last_changed_date']
    }
    leagues.append(league_info)
    
    for team in league['teams']:
        teams.append(team)

# Converting to DataFrames
league_df = pd.DataFrame(leagues)
team_df = pd.DataFrame(teams)

# Merging DataFrames
combined_df = pd.merge(league_df, team_df, on='league_id', how='left', suffixes=('_league', '_team'))

# Displaying the combined DataFrame
print(combined_df)

    league_id                      league_name scoring_type  \
0        5001  Pigskin Prodigal Fantasy League          PPR   
1        5001  Pigskin Prodigal Fantasy League          PPR   
2        5001  Pigskin Prodigal Fantasy League          PPR   
3        5001  Pigskin Prodigal Fantasy League          PPR   
4        5001  Pigskin Prodigal Fantasy League          PPR   
5        5001  Pigskin Prodigal Fantasy League          PPR   
6        5001  Pigskin Prodigal Fantasy League          PPR   
7        5001  Pigskin Prodigal Fantasy League          PPR   
8        5001  Pigskin Prodigal Fantasy League          PPR   
9        5001  Pigskin Prodigal Fantasy League          PPR   
10       5001  Pigskin Prodigal Fantasy League          PPR   
11       5001  Pigskin Prodigal Fantasy League          PPR   
12       5002       Recurring Champions League     Half-PPR   
13       5002       Recurring Champions League     Half-PPR   
14       5002       Recurring Champions League     Half

In [24]:
from tabulate import tabulate

# Flattening the nested data structure
flattened_data = []
for league in data:
    league_info = {
        'league_id': league['league_id'],
        'league_name': league['league_name'],
        'scoring_type': league['scoring_type'],
        'last_changed_date': league['last_changed_date']
    }
    if league['teams']:
        for team in league['teams']:
            team_info = {
                'team_id': team['team_id'],
                'team_name': team['team_name'],
                'team_last_changed_date': team['last_changed_date']
            }
            combined_info = {**league_info, **team_info}
            flattened_data.append(combined_info)
    else:
        flattened_data.append(league_info)

# # Defining the headers
# headers = [
#     'league_id', 'league_name', 'scoring_type', 'last_changed_date',
#     'team_id', 'team_name', 'team_last_changed_date'
# ]

# Printing the data in tabular form
print(tabulate(flattened_data, tablefmt='grid'))

+------+---------------------------------+----------+------------+------+------------------------+------------+
| 5001 | Pigskin Prodigal Fantasy League | PPR      | 2024-04-22 | 1007 | Checkers Not Chess     | 2024-04-22 |
+------+---------------------------------+----------+------------+------+------------------------+------------+
| 5001 | Pigskin Prodigal Fantasy League | PPR      | 2024-04-22 | 1003 | Dallas Steers          | 2024-04-22 |
+------+---------------------------------+----------+------------+------+------------------------+------------+
| 5001 | Pigskin Prodigal Fantasy League | PPR      | 2024-04-22 | 1008 | Defending Champs       | 2024-04-22 |
+------+---------------------------------+----------+------------+------+------------------------+------------+
| 5001 | Pigskin Prodigal Fantasy League | PPR      | 2024-04-22 | 1011 | Double Option          | 2024-04-22 |
+------+---------------------------------+----------+------------+------+------------------------+------